<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0


    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-20 09:28:18
-------------------
qualified stocks: 94
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  1.01 L
Current:  1.38 C
-------------------
Today PnL: -764.00 (-0.01%)
Current PnL: -17.35 L (-11.8%)
CY Booked + Current PnL: -9.08 L (-6.18%)
-------------------
Total profit:  3.95 L
Total loss:  -21.30 L
-------------------
Total Booked + Current PnL: 17.90 L (14.91%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.01%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 76.97 L (55.97%)
Deployed:  1.20 C
Current:  1.38 C
CAGR/XIRR %: 8.69%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LALPATHLAB,2838.53,3545.00,-21.51,H-SC,12.30,234314.0,41294.0,6748.0,1.75,21.39,2.88,24.89,137.0,6.12,1.72,49.73,X40N,NTT,HEALTHCARE
56,MEDANTA,1087.93,1486.00,14.90,H-MC,14.38,164036.0,37836.0,8333.0,2.96,29.98,5.08,36.59,108.0,4.54,1.20,46.25,XY24,NTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.92,112288.0,7976.0,8669.0,-0.69,7.65,7.72,15.96,9.0,0.92,0.82,25.84,X5K,ATH,METALS
69,SBIN,760.30,863.00,-11.34,M-LC,3.85,219712.0,18232.0,8986.0,-0.16,9.05,4.09,13.51,60.0,2.03,1.61,22.79,XY25,NTT,BANKS
4,APOLLOHOSP,7098.95,8285.00,-13.75,H-LC,6.96,172315.0,16138.0,9960.0,0.12,10.33,5.78,16.71,22.0,1.62,1.26,29.57,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-30.43,H-MC,7.42,178470.0,-1338.0,113757.0,-0.39,-0.74,63.74,62.52,88.0,-0.01,1.31,33.09,XR,NTT,BANKS
59,PGHH,13388.00,18062.58,-31.21,H-MC,5.97,199095.0,-1725.0,71853.0,-0.17,-0.86,36.09,34.92,80.0,-0.02,1.46,3.78,X40,ATH,FMCG
57,NATIONALUM,189.63,247.44,-49.46,H-MC,1.95,101394.0,890.0,29749.0,1.58,0.89,29.34,30.49,79.0,0.03,0.74,33.58,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-38.34,H-LC,3.04,148257.0,-1675.0,16264.0,-0.44,-1.12,10.97,9.74,4.0,-0.10,1.09,5.05,X40,NTT,FMCG
21,CIPLA,1492.70,1795.0,-20.81,H-LC,3.26,211350.0,6850.0,34556.0,-0.40,3.35,16.35,20.25,10.0,0.20,1.55,12.38,X40N,BTT,PHARMA
54,LTIM,5564.16,7230.2,-6.94,H-LC,3.70,184590.0,-15720.0,75700.0,0.32,-7.85,41.01,29.94,16.0,-0.21,1.35,27.52,X200,ATH,IT
58,NESTLEIND,1134.30,1377.0,-11.91,H-LC,3.77,271112.0,5686.0,51105.0,-0.24,2.14,18.85,21.40,11.0,0.11,1.99,8.93,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-10.45,H-LC,4.73,174583.0,-26803.0,114422.0,-0.47,-13.31,65.54,43.51,27.0,-0.23,1.28,21.53,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,EASEMYTRIP,18.12,26.40,-4.69,M-SC,26.31,91823.0,-84992.0,165786.0,3.86,-48.07,180.55,45.70,197.0,-0.51,0.67,6.81,XY24,NTT,TRAVEL
56,MEDANTA,1087.93,1486.00,14.90,H-MC,14.38,164036.0,37836.0,8333.0,2.96,29.98,5.08,36.59,108.0,4.54,1.20,46.25,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,13.77,H-MC,5.90,196400.0,31730.0,36236.0,1.82,19.27,18.45,41.28,89.0,0.88,1.44,28.79,X40N,ATH,AC
52,LALPATHLAB,2838.53,3545.00,-21.51,H-SC,12.30,234314.0,41294.0,6748.0,1.75,21.39,2.88,24.89,137.0,6.12,1.72,49.73,X40N,NTT,HEALTHCARE
57,NATIONALUM,189.63,247.44,-49.46,H-MC,1.95,101394.0,890.0,29749.0,1.58,0.89,29.34,30.49,79.0,0.03,0.74,33.58,MH,ATH,METALS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VALIANTORG,512.64,838.00,-318.96,H-SC,15.41,93093.0,-40193.0,124791.0,-1.97,-30.16,134.05,63.47,139.0,-0.32,0.68,56.05,XR,NTT,CHEMICALS
51,KPIGREEN,497.21,731.35,11.32,H-SC,11.31,132678.0,7381.0,51625.0,-1.32,5.89,38.91,47.09,141.0,0.14,0.97,66.06,MH,ATH,POWER
6,ASIANTILES,75.41,137.00,6927.78,L-SC,6.96,77128.0,-16682.0,93302.0,-1.18,-17.78,120.97,81.67,269.0,-0.18,0.56,49.36,XR,NTT,CERAMICS
49,JIOFIN,310.58,387.00,3.79,H-LC,14.97,229530.0,12124.0,41361.0,-1.03,5.58,18.02,24.61,48.0,0.29,1.68,63.45,XY24,BTT,FINANCE
43,INDIGOPNTS,1407.73,1408.00,119.31,M-SC,1.95,143790.0,-30769.0,30800.0,-1.01,-17.63,21.42,0.02,221.0,-1.00,1.05,24.69,OX40N,NTT,PAINTS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,TTKPRESTIG,769.29,770.0,100.64,M-SC,9.60,86480.0,-14297.0,14390.0,-0.53,-14.19,16.64,0.09,245.0,-0.99,0.63,13.62,OX40N,NTT,DURABLES
43,INDIGOPNTS,1407.73,1408.0,119.31,M-SC,1.95,143790.0,-30769.0,30800.0,-1.01,-17.63,21.42,0.02,221.0,-1.00,1.05,24.69,OX40N,NTT,PAINTS
20,CERA,7989.07,9475.0,-17.90,H-SC,3.00,116991.0,-26812.0,53558.0,0.02,-18.65,45.78,18.60,149.0,-0.50,0.86,29.10,OX40N,NTT,CERAMICS
85,UJJIVANSFB,52.77,60.0,83.20,H-SC,16.45,115722.0,-26757.0,46277.0,-0.23,-18.78,39.99,13.70,163.0,-0.58,0.85,36.58,OX40N,NTT,BANKS
50,KANSAINER,299.63,340.0,-68.52,H-SC,3.26,216576.0,-53091.0,89424.0,-0.92,-19.69,41.29,13.47,138.0,-0.59,1.59,9.54,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TCS,3794.03,4998.00,-28.69,H-LC,16.25,236098.0,-59836.0,153747.0,0.35,-20.22,65.12,31.73,1.0,-0.39,1.73,0.80,X40,BTT,IT
45,INFY,1461.46,2275.00,-20.32,H-LC,11.88,261143.0,-458.0,146083.0,1.31,-0.18,55.94,55.67,3.0,-0.00,1.91,5.92,X40,BTT,IT
47,ITC,411.90,452.00,-38.34,H-LC,3.04,148257.0,-1675.0,16264.0,-0.44,-1.12,10.97,9.74,4.0,-0.10,1.09,5.05,X40,NTT,FMCG
90,VBL,492.64,671.64,-9.95,H-LC,9.65,273600.0,8560.0,87744.0,0.62,3.23,32.07,36.33,5.0,0.10,2.00,16.96,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.62,H-LC,11.66,198120.0,-18399.0,111304.0,0.24,-8.50,56.18,42.91,7.0,-0.17,1.45,3.76,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LALPATHLAB,2838.53,3545.0,-21.51,H-SC,12.30,234314.0,41294.0,6748.0,1.75,21.39,2.88,24.89,137.0,6.12,1.72,49.73,X40N,NTT,HEALTHCARE
56,MEDANTA,1087.93,1486.0,14.90,H-MC,14.38,164036.0,37836.0,8333.0,2.96,29.98,5.08,36.59,108.0,4.54,1.20,46.25,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-8.01,H-LC,7.93,229766.0,45285.0,25090.0,0.77,24.55,10.92,38.14,38.0,1.80,1.68,37.75,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,MEDANTA,1087.93,1486.00,14.90,H-MC,14.38,164036.0,37836.0,8333.0,2.96,29.98,5.08,36.59,108.0,4.54,1.20,46.25,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,13.77,H-MC,5.90,196400.0,31730.0,36236.0,1.82,19.27,18.45,41.28,89.0,0.88,1.44,28.79,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,11.32,H-SC,11.31,132678.0,7381.0,51625.0,-1.32,5.89,38.91,47.09,141.0,0.14,0.97,66.06,MH,ATH,POWER
91,VOLTAS,1278.28,1918.49,-1.64,H-MC,7.38,206460.0,14718.0,81304.0,-0.18,7.68,39.38,50.08,99.0,0.18,1.51,15.00,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-3.47,H-MC,10.44,179354.0,28441.0,28858.0,0.12,18.85,16.09,37.97,93.0,0.99,1.31,48.92,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.47,H-MC,10.44,179354.0,28441.0,28858.0,0.12,18.85,16.09,37.97,93.0,0.99,1.31,48.92,AR,ATH,AUTO
51,KPIGREEN,497.21,731.35,11.32,H-SC,11.31,132678.0,7381.0,51625.0,-1.32,5.89,38.91,47.09,141.0,0.14,0.97,66.06,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-50.53,H-SC,11.98,138081.0,14745.0,119012.0,0.61,11.96,86.19,108.45,119.0,0.12,1.01,36.98,AR,ATH,MISC
92,WHIRLPOOL,1219.98,2270.00,-42.31,M-SC,4.98,97575.0,6076.0,72674.0,1.23,6.64,74.48,86.07,223.0,0.08,0.71,41.15,XR,NTT,DURABLES
62,RAJOOENG,102.50,143.33,-31.07,H-SC,8.04,104400.0,1900.0,38931.0,1.20,1.85,37.29,39.83,114.0,0.05,0.76,3.37,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.79,75763.0,-37786.0,77839.0,0.10,-33.28,102.74,35.27,268.0,-0.49,0.55,85.43,XR,NTT,HOTELS
6,ASIANTILES,75.41,137.00,6927.78,L-SC,6.96,77128.0,-16682.0,93302.0,-1.18,-17.78,120.97,81.67,269.0,-0.18,0.56,49.36,XR,NTT,CERAMICS
3,ALKYLAMINE,2347.98,4546.37,-3.33,H-SC,15.58,89990.0,-10973.0,105504.0,-0.54,-10.87,117.24,93.63,148.0,-0.10,0.66,34.04,SR,ATH,CHEMICALS
55,MASFIN,326.60,399.50,-17.34,H-SC,12.36,93870.0,-4110.0,25983.0,-0.14,-4.19,27.68,22.32,152.0,-0.16,0.69,36.14,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-10.68,H-LC,7.01,95947.0,-1798.0,39194.0,-0.71,-1.84,40.85,38.26,35.0,-0.05,0.70,25.64,X40,BTT,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.47,H-MC,10.44,179354.0,28441.0,28858.0,0.12,18.85,16.09,37.97,93.0,0.99,1.31,48.92,AR,ATH,AUTO
28,DMART,3764.92,5201.00,-8.01,H-LC,7.93,229766.0,45285.0,25090.0,0.77,24.55,10.92,38.14,38.0,1.80,1.68,37.75,X40N,NTT,FMCG
52,LALPATHLAB,2838.53,3545.00,-21.51,H-SC,12.30,234314.0,41294.0,6748.0,1.75,21.39,2.88,24.89,137.0,6.12,1.72,49.73,X40N,NTT,HEALTHCARE
56,MEDANTA,1087.93,1486.00,14.90,H-MC,14.38,164036.0,37836.0,8333.0,2.96,29.98,5.08,36.59,108.0,4.54,1.20,46.25,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,7098.95,8285.00,-13.75,H-LC,6.96,172315.0,16138.0,9960.0,0.12,10.33,5.78,16.71,22.0,1.62,1.26,29.57,X40N,BTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.48
1,25,40.99
2,50,70.50


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.53
LC    35.36
MC    26.07
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.15
X40N     15.38
X40      14.07
XY25     11.59
XR        8.82
AR        8.28
OX40N     7.33
X5K       2.18
MH        1.71
X200      1.35
SR        1.10
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.25
H-SC    23.98
H-MC    23.21
M-SC    13.08
M-LC     7.07
M-MC     2.53
L-SC     1.47
L-LC     1.04
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.18,-0.14,30.81
IT,11.58,-18.08,77.06
FINANCE,7.99,-19.47,63.67
BANKS,7.44,-12.28,59.96
MISC,5.71,-17.66,85.10
PAINTS,5.55,-15.28,38.00
HEALTHCARE,5.20,7.47,15.89
ELECTRICAL,4.65,-10.64,58.56
AUTO,4.52,-9.33,56.90


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2791318.0,23
XR,1088523.0,13
AR,1076010.0,10
X40,816663.0,11
X40N,507902.0,12
XY25,466922.0,8
OX40N,458761.0,10
SR,193685.0,2
X5K,140349.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2744157.0,28
M-SC,1456973.0,17
H-MC,1211388.0,18
H-LC,1178759.0,19
M-LC,411940.0,5
M-MC,331969.0,2
L-SC,259605.0,3
L-MC,57868.0,1
L-LC,44548.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,927451.0,6
M-SC,XY24,855581.0,7
H-SC,AR,657082.0,5
H-LC,X40,528109.0,6
H-SC,XR,517829.0,6
H-MC,XY24,470994.0,5
M-MC,XY24,331969.0,2
H-LC,AR,232254.0,2
H-MC,X40,208687.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
